In [18]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.autograd import Variable
from torch import Tensor, optim, nn
import wandb
from tqdm import tqdm
from sklearn.metrics import classification_report

wandb.login()

True

In [19]:
config = {
    "learning_rate": 0.01,
    "architecture": "GRU",
    "dataset": "timeline_1.0",
    "epochs": 50,
    "classes": 2,
    "batch_size": 128,
    "num_layers": 2,
    "hidden_size": 64,
    "dropout_prob": 0.5,
    "input_size": 381,
    "output_size": 1,
    "optimizer": "Adam",
    "loss": "CrossEntropyLoss",
    "activation": "ReLU",
    "initializer": "Xavier",
    "regularization": "L2",
    "regularization_lambda": 0.01,
    "gru_layers": 1,
    "sequence_length": 16,

}

In [20]:
def model_pipeline(hyperparameters):
    with wandb.init(project="leaguify", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, val_loader, test_loader, criterion, optimizer = make(config)
        print(model)

        # and use them to train the model
        train(model, train_loader, val_loader, criterion, optimizer, config)

        # and test its final performance
        test(model, val_loader, 'val')
        test(model, test_loader, 'test')
    return model

In [21]:
class TimelineDataset(Dataset):
    def __init__(self, data_dir, sequence_length, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, device=device)
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64, device=device)
        self.sequence_length = sequence_length
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        sample = self.data[idx:idx + self.sequence_length, :]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

In [22]:
def make(config):
    train = get_train_data(sequence_length=config.sequence_length)
    val = get_val_data(sequence_length=config.sequence_length)
    test = get_test_data(sequence_length=config.sequence_length)
    train_loader = make_loader(train, batch_size=config.batch_size)
    val_loader = make_loader(val, batch_size=config.batch_size)
    test_loader = make_loader(test, batch_size=config.batch_size)

    model = GRU(config.input_size, config.hidden_size, config.classes, config.num_layers, config
                .gru_layers, drop_prob=config.dropout_prob).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    return model, train_loader, val_loader, test_loader, criterion, optimizer

In [23]:
def get_train_data(sequence_length=16) -> (torch.utils.data.Dataset, torch.utils.data.Dataset):
    dataset = TimelineDataset('../data/processed/train_timeline.npy', sequence_length)
    return dataset

In [24]:
def get_val_data(sequence_length=16):
    """
    Get the validation data
    :param sequence_length: 
    :return: 
    """
    dataset = TimelineDataset('../data/processed/val_timeline.npy', sequence_length)
    return dataset

In [25]:
def check_correct_split(full_dataset, train_data, val_data, sequence_length=16):
    """
    Check that the train/val split is correct and no data is leaked between the two
    appends both datasets together, removes duplicates and checks that the original dataset is the same as the combined
    :param full_dataset: 
    :param train_data: 
    :param val_data: 
    :param sequence_length: 
    :return: 
    """
    len_combined = len(full_dataset)
    full_no_dup = torch.unique(full_dataset, dim=0)
    len_no_dup = len(full_no_dup)
    original = TimelineDataset('../data/processed/train_timeline.npy', sequence_length)
    len_original = len(original)
    print(f'len_combined: {len_combined}, len_no_dup: {len_no_dup}, len_original: {len_original}')
    assert len_combined == len_no_dup
    assert len_original == len_combined



In [26]:
def get_test_data(sequence_length=16):
    full_dataset = TimelineDataset('../data/processed/test_timeline.npy', sequence_length)
    return full_dataset

In [27]:
def make_loader(dataset, batch_size=64):
    return DataLoader(dataset, batch_size=batch_size, num_workers=0, drop_last=False)

In [28]:
device = (
    "cuda" if torch.cuda.is_available()
    else "cpu"
)
if torch.cuda.is_available():
    print(f'PyTorch version: {torch.__version__}')
    print('*' * 10)
    print(f'_CUDA version: ')
    !nvcc --version
    print('*' * 10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
print(f"Using {device} device")

PyTorch version: 2.1.0+cu121
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
**********
CUDNN version: 8801
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 2080
Using cuda device


In [29]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, gru_layers, drop_prob=0.2):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(input_dim, hidden_dim, gru_layers, batch_first=True, dropout=drop_prob)
        self.gru2 = nn.GRU(hidden_dim, hidden_dim, gru_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, h=None):
        out, h = self.gru(x, h)
        out, h = self.gru2(out, h)
        out = self.fc(self.relu(out[:, -1]))
        return out, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(1, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [30]:
def train(model, train_loader, val_loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log='all', log_freq=10)

    total_batches = len(train_loader) * config.epochs
    example_count = 0
    batch_count = 0
    loss_vals = []
    for epoch in tqdm(range(config.epochs)):
        print(f'Epoch: {epoch}')
        tested = False
        h = model.init_hidden(config.batch_size)
        for _, (matches, labels) in enumerate(train_loader):
            model.train()
            output, h = model(matches)  # hidden state is not passed to re-init at each batch
            loss = criterion(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            example_count += len(matches)
            batch_count += 1
            if (batch_count + 1) % 25 == 0:
                if not tested:
                    train_log(loss, example_count, epoch, val_loader, model, to_test=True, train_loader=train_loader)
                    tested = True
                else:
                    train_log(loss, example_count, epoch, val_loader, model, to_test=False)

In [31]:
def train_log(loss, example_count, epoch, val_loader, model, to_test=False, train_loader=None):
    if to_test:
        val_acc = test(model, val_loader, 'val')
        train_acc = test(model, train_loader, 'train')
        wandb.log({"epoch": epoch, "val_accuracy": val_acc, "train_accuracy": train_acc}, step=example_count)
    wandb.log({"epoch": epoch, "train_loss": loss}, step=example_count)
    print(f"Loss after {str(example_count).zfill(5)} examples: {loss:.3f}")

In [32]:
train_data = get_train_data(sequence_length=16)
val_data = get_val_data(sequence_length=16)
print(f'train_data: {len(train_data)}')
print(f'val_data: {len(val_data)}')
for matches, labels in make_loader(val_data, batch_size=1):
    if matches.shape[1] != 16:
        print(f'matches: {matches.shape}, labels: {labels.shape}')

train_data: 38128
val_data: 12688


In [33]:
def test(model, loader, loader_name='val'):
    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        y_pred, y_true = [], []
        for matches, labels in loader:
            matches, labels = matches.to(device), labels.to(device)
            model.eval()
            output, h = model(matches)
            _, predicted = torch.max(output, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(classification_report(y_true, y_pred))
        print(f"Accuracy of the model on the {total} " +
              f"{loader_name} matches: {correct / total:%}")

        wandb.log({f"{loader_name}_accuracy": correct / total})
    return correct / total

In [34]:
model = model_pipeline(config)

C:\Users\morit\AppData\Local\pypoetry\Cache\virtualenvs\leaguify-VaCbhr8h-py3.11\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


GRU(
  (gru): GRU(381, 64, batch_first=True, dropout=0.5)
  (gru2): GRU(64, 64, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0
              precision    recall  f1-score   support

           0       0.55      0.97      0.70      6928
           1       0.51      0.04      0.07      5760

    accuracy                           0.55     12688
   macro avg       0.53      0.50      0.38     12688
weighted avg       0.53      0.55      0.41     12688

Accuracy of the model on the 12688 val matches: 54.657945%
              precision    recall  f1-score   support

           0       0.52      0.97      0.68     19792
           1       0.59      0.05      0.09     18336

    accuracy                           0.53     38128
   macro avg       0.55      0.51      0.39     38128
weighted avg       0.55      0.53      0.40     38128

Accuracy of the model on the 38128 train matches: 52.622744%
Loss after 03072 examples: 0.726
Loss after 06272 examples: 0.669
Loss after 09472 examples: 0.682
Loss after 12672 examples: 0.650
Loss after 15872 examples: 0.720
Loss after 19072 examples: 0.620
Loss after 22272 ex

  2%|▏         | 1/50 [00:02<02:26,  3.00s/it]

...

Loss after 1864704 examples: 0.113
Loss after 1867904 examples: 0.040
Epoch: 49
              precision    recall  f1-score   support

           0       0.71      0.65      0.68      6928
           1       0.62      0.68      0.65      5760

    accuracy                           0.67     12688
   macro avg       0.66      0.67      0.66     12688
weighted avg       0.67      0.67      0.67     12688

Accuracy of the model on the 12688 val matches: 66.511665%
              precision    recall  f1-score   support

           0       0.94      0.90      0.92     19792
           1       0.90      0.93      0.92     18336

    accuracy                           0.92     38128
   macro avg       0.92      0.92      0.92     38128
weighted avg       0.92      0.92      0.92     38128

Accuracy of the model on the 38128 train matches: 91.722619%
Loss after 1871088 examples: 0.360
Loss after 1874288 examples: 0.140
Loss after 1877488 examples: 0.151
Loss after 1880688 examples: 0.149
Loss a

100%|██████████| 50/50 [02:32<00:00,  3.05s/it]

Loss after 1903088 examples: 0.150
Loss after 1906288 examples: 0.188


              precision    recall  f1-score   support

           0       0.70      0.63      0.67      6928
           1       0.61      0.68      0.64      5760

    accuracy                           0.66     12688
   macro avg       0.66      0.66      0.65     12688
weighted avg       0.66      0.66      0.66     12688

Accuracy of the model on the 12688 val matches: 65.526482%
              precision    recall  f1-score   support

           0       0.75      0.66      0.70      7168
           1       0.62      0.71      0.66      5520

    accuracy                           0.68     12688
   macro avg       0.68      0.68      0.68     12688
weighted avg       0.69      0.68      0.68     12688

Accuracy of the model on the 12688 test matches: 68.024905%


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_accuracy,▁
train_accuracy,▁▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██▇██▇▇████████▇███████
train_loss,▇▆█▅█▅▆▇▆▃▃▃▂▃▅▃▂▄▂▁▂▁▃▃▅▃▂▂▂▂▄▃▃▂▄▂▂▃▅▁
val_accuracy,▁▆▇▆▇▇▇▇▇▇▇█▆▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▇▇
epoch,49
test_accuracy,0.68025
train_accuracy,0.91723
train_loss,0.18833
val_accuracy,0.65526
